# SIGIR eCOM 2021 Data Challenge

- Coveo
    - [GitHub](https://github.com/coveooss/SIGIR-ecom-data-challenge) de la compétition.
    - https://arxiv.org/pdf/2104.09423.pdf
    - Les données sont disponibles ici: https://www.coveo.com/en/ailabs/sigir-ecom-data-challenge
- La solution de l'équipe NVIDIA
    - [Winning the SIGIR eCommerce Challenge on session-based recommendation with Transformers](https://medium.com/nvidia-merlin/winning-the-sigir-ecommerce-challenge-on-session-based-recommendation-with-transformers-v2-793f6fac2994
    - [GitHub](https://github.com/NVIDIA-Merlin/competitions/tree/main/SIGIR_eCommerce_Challenge_2021).  Malheureusement ce dépôt GitHub ne contient pas le fichier `test/rec_test_phase_1.json` pour créer le fichier test. On ne peut donc pas reproduire les calculs précis de l'équipe NVIDIA.
    - [Paper](https://arxiv.org/abs/2107.05124)
- Transformers4Rec
  - NVIDIA, [Transformers4Rec](https://github.com/NVIDIA-Merlin/Transformers4Rec) sur GitHub
  - [Tutorial](https://nvidia-merlin.github.io/Transformers4Rec/main/examples/tutorial/index.html)

In [83]:
import os 
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

## How to Start

Le code de cette section provient de [How to Start](https://github.com/coveooss/SIGIR-ecom-data-challenge/blob/main/README_DC_2021.md#how-to-start).

Download the `zip` folder and unzip it in your local machine. To verify that all is well, you can run the simple `start/dataset_stats.py` script in the folder: the script will parse the three files, show some sample rows and print out some basic stats and counts (if you don't modify the three paths, it will run on the sample `csv`).

In [2]:
PATH = "/Users/alain/data/SIGIR-ecom-data-challenge"

In [3]:
import csv
from datetime import datetime

# put here the file paths if you did not unzip in same folder
BROWSING_FILE_PATH = os.path.join(PATH, 'start/browsing_train_sample.csv')
SEARCH_TRAIN_PATH = os.path.join(PATH, 'start/search_train_sample.csv')
SKU_2_CONTENT_PATH = os.path.join(PATH, 'start/sku_to_content_sample.csv')


def get_rows(file_path: str, print_limit: int = 2):
    """
    Util function reading the csv file and printing the first few lines out for visual debugging.

    :param file_path: local path to the csv file
    :param print_limit: specifies how many rows to print out in the console for debug
    :return: list of dictionaries, one per each row in the file
    """
    rows = []
    print("\n============== {}".format(file_path))
    with open(file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for idx, row in enumerate(reader):
            # print out first few lines
            if idx < print_limit:
                print(row)
            rows.append(row)

    return rows


def get_descriptive_stats(
        browsing_train_path : str,
        search_train_path: str,
        sku_2_content_path: str
):
    """
    Simple function showing how to read the main training files, print out some
    example rows, and producing the counts found in the Data Challenge paper.

    We use basic python library commands, optimizing for clarity, not performance.

    :param browsing_train_path: path to the file containing the browsing interactions
    :param search_train_path: path to the file containing the search interactions
    :param sku_2_content_path: path to the file containing the product meta-data
    :return:
    """
    print("Starting our counts at {}".format(datetime.utcnow()))
    # first, just read in the csv files and display some rows
    browsing_events = get_rows(browsing_train_path)
    print("# {} browsing events".format(len(browsing_events)))
    search_events = get_rows(search_train_path)
    print("# {} search events".format(len(search_events)))
    sku_mapping = get_rows(sku_2_content_path)
    print("# {} products".format(len(sku_mapping)))
    # now do some counts
    print("\n\n=============== COUNTS ===============")
    print("# {} of distinct SKUs with interactions".format(
        len(set([r['product_sku_hash'] for r in browsing_events if r['product_sku_hash']]))))
    print("# {} of add-to-cart events".format(sum(1 for r in browsing_events if r['product_action'] == 'add')))
    print("# {} of purchase events".format(sum(1 for r in browsing_events if r['product_action'] == 'purchase')))
    print("# {} of total interactions".format(sum(1 for r in browsing_events if r['product_action'])))
    print("# {} of distinct sessions".format(
        len(set([r['session_id_hash'] for r in browsing_events if r['session_id_hash']]))))
    # now run some tests
    print("\n\n*************** TESTS ***************")
    for r in browsing_events:
        assert len(r['session_id_hash']) == 64
        assert not r['product_sku_hash'] or len(r['product_sku_hash']) == 64
    for p in sku_mapping:
        assert not p['price_bucket'] or float(p['price_bucket']) <= 10
    # say goodbye
    print("All done at {}: see you, space cowboy!".format(datetime.utcnow()))

    return


if __name__ == "__main__":
    get_descriptive_stats(
        BROWSING_FILE_PATH,
        SEARCH_TRAIN_PATH,
        SKU_2_CONTENT_PATH
    )

Starting our counts at 2023-10-15 17:01:46.477128

============== /Users/alain/data/SIGIR-ecom-data-challenge/start/browsing_train_sample.csv
{'session_id_hash': '20c458b802f6ea9374783bfc528b19421be977a6769785ec2b357915dd6fda84', 'event_type': 'event_product', 'product_action': 'detail', 'product_sku_hash': 'd5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f8eafbc503f4012f69c', 'server_timestamp_epoch_ms': '1550885210881', 'hashed_url': '7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d2bb8b99951799f45ea'}
{'session_id_hash': '20c458b802f6ea9374783bfc528b19421be977a6769785ec2b357915dd6fda84', 'event_type': 'event_product', 'product_action': 'detail', 'product_sku_hash': '61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d516576eb808a0904dc3', 'server_timestamp_epoch_ms': '1550885213307', 'hashed_url': '4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce44dc007eaed41f4ad9'}
# 34 browsing events

============== /Users/alain/data/SIGIR-ecom-data-challenge/start/search_train_sample.csv
{'session_id_hash': '

## Loading Train Data

See https://github.com/NVIDIA-Merlin/competitions/blob/main/SIGIR_eCommerce_Challenge_2021/task1_session_based_rec/0-eda/EDA_train_x_test_set.ipynb

### Browsing Events

Le fichier `browsing_train.csv` contient près de 5M de sessions d'achat anonymisées [sessions](https://support.google.com/analytics/answer/2731565?hl=fr).
La structure de cet ensemble de données est similaire à notre publication de données [Scientific Reports](https://github.com/coveooss/shopper-intent-prediction-nature-2020) :
chaque ligne correspond à un événement de navigation dans une session, contenant des informations sur la session et l'horodatage, ainsi que
des détails (hachés) sur l'interaction (était-ce un événement _achat_ ou un événement _détail_ ? Était-ce une simple _page vue_ ou une action
de produit spécifique ?). Toutes les données ont été collectées et traitées de manière anonyme grâce à notre [SDK](https://docs.coveo.com/en/3188/coveo-for-commerce/tracking-commerce-events) standard :
n'oubliez pas que le suivi en front-end est par nature imparfait, donc de petites incohérences sont à prévoir.

Champ | Type | Description
------------ | ------------- | -------------
session_id_hash | chaîne de caractères | Identifiant haché de la session d'achat. Une session regroupe des événements qui sont espacés d'au maximum 30 minutes : si le même utilisateur revient sur le site cible après 31 minutes depuis la dernière interaction, un nouvel identifiant de session est attribué.
event_type | énumération | Le type d'événement selon le [Protocole Google](https://developers.google.com/analytics/devguides/collection/protocol/v1), l'un des { _pageview_ , _event_ } ; par exemple, un événement _add_ peut se produire lors du chargement d'une page ou en tant qu'événement autonome.
product_action | énumération | L'un des { _detail_, _add_, _purchase_, _remove_ }. Si le champ est vide, l'événement est une simple vue de page (par exemple, la page `FAQ`) sans produits associés. Veuillez également noter qu'une action impliquant la suppression d'un produit du panier peut entraîner plusieurs événements _remove_ consécutifs. Veuillez noter que les événements _click_ (c'est-à-dire les événements générés en cliquant sur une page de recherche) sont inclus dans le fichier `search_train.csv`.
product_sku_hash | chaîne de caractères | Si l'événement est un événement _product_, identifiant haché du produit dans l'événement.
server_timestamp_epoch_ms | entier | Temps d'époque, en millisecondes. Comme technique d'anonymisation supplémentaire, l'horodatage a été décalé d'un nombre de semaines non spécifié, en préservant les modèles intra-semaine.
hashed_url | chaîne de caractères | URL hachée de la page web actuelle.

Enfin, veuillez noter qu'un PDP peut générer à la fois un événement _detail_ et un événement _pageview_, et que l'ordre des événements dans le
fichier n'est pas strictement chronologique (reportez-vous à l'identifiant de session et aux informations d'horodatage pour reconstruire la
chaîne réelle des événements pour une session donnée).


In [87]:
train_browsing_df = pd.read_csv(os.path.join(PATH, 'train/browsing_train.csv'))
train_browsing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36079307 entries, 0 to 36079306
Data columns (total 6 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   session_id_hash            object
 1   event_type                 object
 2   product_action             object
 3   product_sku_hash           object
 4   server_timestamp_epoch_ms  int64 
 5   hashed_url                 object
dtypes: int64(1), object(5)
memory usage: 1.6+ GB


In [5]:
train_browsing_df

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
2,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
3,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
4,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
...,...,...,...,...,...,...
36079302,0676f342dc490b0f8bd9c22d16e4c67f8f7af1f85679f1...,pageview,NaN,NaN,1552162324909,38f5bd3c9a1cc5b39e6b965f1aa6c565737f58e19a560a...
36079303,0676f342dc490b0f8bd9c22d16e4c67f8f7af1f85679f1...,pageview,NaN,NaN,1552162336608,38f5bd3c9a1cc5b39e6b965f1aa6c565737f58e19a560a...
36079304,0676f342dc490b0f8bd9c22d16e4c67f8f7af1f85679f1...,pageview,NaN,NaN,1552162343684,38f5bd3c9a1cc5b39e6b965f1aa6c565737f58e19a560a...
36079305,0676f342dc490b0f8bd9c22d16e4c67f8f7af1f85679f1...,pageview,NaN,NaN,1552162356368,433b0e71df1fe9a8d1f45647545701f6108414c40eef76...


In [88]:
# Fréquence des types d'événements.
train_browsing_df['event_type'].value_counts()

event_type
pageview         25647696
event_product    10431611
Name: count, dtype: int64

In [89]:
# Fréquence des actions sur les produits.
train_browsing_df['product_action'].value_counts()

product_action
detail      9707890
add          329557
remove       316316
purchase      77848
Name: count, dtype: int64

### Search Events

Le fichier `search_train.csv` contient plus de 800 000 interactions basées sur la recherche. Chaque ligne est un événement de requête de recherche effectué par un acheteur, qui comprend un ensemble de résultats (hachés) renvoyés au client. Nous indiquons également quel(s) résultat(s) ont été cliqués dans l'ensemble des résultats, le cas échéant.
En signalant également les produits vus mais non cliqués, nous espérons inspirer des moyens astucieux d'utiliser les retours négatifs.

Champ | Type | Description
------------ | ------------- | -------------
session_id_hash | chaîne de caractères | Identifiant haché de la session d'achat. Une session regroupe des événements qui sont au maximum à 30 minutes d'intervalle : si le même utilisateur revient sur le site cible après 31 minutes depuis la dernière interaction, un nouvel identifiant de session est attribué.
server_timestamp_epoch_ms | entier | Heure d'époque, en millisecondes. En tant que technique d'anonymisation supplémentaire, l'horodatage a été décalé d'un nombre non spécifié de semaines, tout en préservant les modèles intra-semaine.
query_vector | vecteur | Une représentation dense de la requête de recherche, obtenue grâce à des techniques standard de modélisation pré-entraînée et de réduction de dimensionnalité.
product_skus_hash | liste | Identifiants hachés des produits dans la réponse de recherche.
clicked_skus_hash | liste | Identifiants hachés des produits sur lesquels on a cliqué après avoir émis la requête de recherche.



In [90]:
train_search_df = pd.read_csv(os.path.join(PATH, 'train/search_train.csv'))
train_search_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 819516 entries, 0 to 819515
Data columns (total 5 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   session_id_hash            819516 non-null  object
 1   query_vector               819516 non-null  object
 2   clicked_skus_hash          179495 non-null  object
 3   product_skus_hash          602754 non-null  object
 4   server_timestamp_epoch_ms  819516 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 31.3+ MB


In [93]:
train_search_df

,session_id_hash,query_vector,clicked_skus_hash,product_skus_hash,server_timestamp_epoch_ms
0,48fade624d47870058ce07dd789ccc04e46c70c0fa2a1b...,"[-0.20255649089813232, -0.016908567398786545, ...",NaN,NaN,1548575194779
1,8731ca84ff7bb8cb647531d54e64feedb2519b4a7792a7...,"[-0.007610442116856575, -0.14909175038337708, ...",NaN,['9ee9ffd7e2529a65f9a0b0c9eaae6330df85cf2e3af3...,1548276763869
2,9be980708345944960645d03606ea83b637cae9106b705...,"[-0.20023074746131897, -0.03151938319206238, 0...",NaN,['7cc72dbed53bab78ec6a62feaa5052a7a1db7d201664...,1548937997295
3,9be980708345944960645d03606ea83b637cae9106b705...,"[-0.18556387722492218, -0.07620412111282349, 0...",NaN,['62c4ddab6c1c81c74d315376b3c0dc7768c0286b3dc6...,1548938038268
4,9be980708345944960645d03606ea83b637cae9106b705...,"[-0.03269264101982117, -0.27234694361686707, 0...",NaN,['2a0ee2924feabeec35e21e8fcb4d5b0684d190e46cef...,1548938093827
...,...,...,...,...,...
819511,9b037012702a2dadee910c6b71b691b6ba072b22f41b7b...,"[-0.04411918669939041, -0.15845978260040283, -...",['f023b48bf46355aa4844b8c9bd51e478a22f17b9be83...,['12db169815cb93aaeeb1d427d6c0eea62ba4961e43c4...,1548987598321
819512,9b037012702a2dadee910c6b71b691b6ba072b22f41b7b...,"[-0.033466748893260956, 0.06939797103404999, -...",['1379461daf84128449e010af5f2a0d37837bac53c854...,['956c335d2f59c1853f499420ae4e85250a6c7190a1df...,1548987688646
819513,fdf03b124f6cb368c42a9408bc1186e4ea1465cacd5855...,"[-0.04076826199889183, -0.3194904327392578, 0....",NaN,NaN,1548554188376
819514,fdf03b124f6cb368c42a9408bc1186e4ea1465cacd5855...,"[0.0175111535936594, -0.2686958312988281, 0.07...",NaN,['32ead81a7b89168dc472d162a3d7654db02d99bb88b3...,1548554908239


In [6]:
# Timestamp minimum et maximum.
train_browsing_df['server_timestamp_epoch_ms'].agg(['min', 'max'])

min    1547528564513
max    1555300798560
Name: server_timestamp_epoch_ms, dtype: int64

In [91]:
# Nombre de produits pour lesquels on a au moins un clique.
train_search_items_clicked = train_search_df[~train_search_df['clicked_skus_hash'].isna()]['clicked_skus_hash'].explode().unique()
len(train_search_items_clicked)

73311

In [97]:
# Test pour voir ce que fait explode().unique()
c = [[1, 2, 3], [2, 3, 4], [5, 6, 7]]
df = pd.DataFrame({'c': c})
print(df)
print(df['c'].explode())
print(df['c'].explode().unique())

           c
0  [1, 2, 3]
1  [2, 3, 4]
2  [5, 6, 7]
0    1
0    2
0    3
1    2
1    3
1    4
2    5
2    6
2    7
Name: c, dtype: object
[1 2 3 4 5 6 7]


### Catalog Metadata

Le fichier `sku_to_content.csv` contient une correspondance entre les identifiants de produit (SKU) hachés et la représentation dense
des métadonnées textuelles et d'image du catalogue réel, pour tous les SKU dans l'ensemble d'entraînement et l'ensemble d'évaluation du Challenge
(lorsque l'information est disponible).

Champ | Type | Description
------------ | ------------- | -------------
product_sku_hash | chaîne de caractères | Identifiant haché du produit (SKU).
category_hash | chaîne de caractères | Les catégories sont des représentations hachées de la hiérarchie des catégories, séparées par un `/`.
price_bucket | entier | Le prix du produit, fourni sous forme d'entier quantile à 10 niveaux.
description_vector | vecteur | Une représentation dense des métadonnées textuelles, obtenue grâce à des techniques standard de modélisation pré-entraînée et de réduction de la dimensionalité. Veuillez noter que cette représentation est compatible avec celle dans le fichier de recherche.
image_vector| vecteur | Une représentation dense des métadonnées d'image, obtenue grâce à des techniques standard de modélisation pré-entraînée et de réduction de la dimensionalité.


In [92]:
# train_sku_to_content_df = pd.read_csv(os.path.join(PATH, 'train/sku_to_content.csv'))
train_sku_to_content_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66386 entries, 0 to 66385
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_sku_hash    66386 non-null  object 
 1   description_vector  31950 non-null  object 
 2   category_hash       32052 non-null  object 
 3   image_vector        28370 non-null  object 
 4   price_bucket        32038 non-null  float64
dtypes: float64(1), object(4)
memory usage: 2.5+ MB


In [116]:
train_sku_to_content_df

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket
0,26ce7b47f4c46e4087e83e54d2f7ddc7ea57862fed2e2a...,NaN,NaN,NaN,NaN
1,6383992be772b204a9ab75f86c86f5583d1bdd1222952d...,NaN,NaN,NaN,NaN
2,a2c3e2430c6ef9770b903ad08fa067a6b2b9db28f06e1b...,"[0.27629122138023376, -0.15763211250305176, 0....",06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,"[340.3592564184389, -220.19025864725685, 154.0...",7.0
3,1028ef615e425c328e7b95010dfb1fb93cf63749a1bc80...,"[0.4058118760585785, -0.03595402091741562, 0.2...",115a6a7017ee55752b8487c77dfde92b0d501d10a2e69c...,"[180.3463662921092, 222.702322343354, -8.88703...",8.0
4,9870c682d0d52d635501249da0eeaa118fad430b695ea1...,"[-0.3206155300140381, 0.01991105079650879, 0.0...",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[-114.81079301576219, 84.55770104232334, 85.51...",2.0
...,...,...,...,...,...
66381,c7cc673ca3baa5fa222fffdc16379892b3a62583a48143...,NaN,NaN,NaN,NaN
66382,6641c7d2053ce48ce1e81a9653dffe56dbb79ab0704fbd...,"[-0.19150441884994507, -0.06235162168741226, -...",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[129.55668732976045, 43.27996741934932, -36.70...",3.0
66383,526a6a51717d5bb40ef2b0c47394d08c54385375633bab...,"[-0.19760936498641968, 0.4446450471878052, -0....",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[-158.25984189321855, 74.19255741438077, 199.6...",5.0
66384,21ca4ab0e2fbd3b401fbeadeb4439dcab9998fb52159ec...,NaN,NaN,NaN,NaN


**NOTE** : Bizarrement, nous avons moins de produits (66386) que le nombre de produits différents pour lesquels nous avons au moins un clique (73311).

## Test set

In [10]:
import json 

with open(os.path.join(PATH, 'baselines/session_rec_sigir_data/test/rec_test_sample.json')) as json_file:
    test_queries = json.load(json_file)
    testset_recommendation_df = pd.json_normalize(test_queries, 'query')
    print(len(testset_recommendation_df))

59


**NOTE** : Pas le même fichier json que NVIDIA. J'obtiens 59 lignes tandis que le fichier de NVIDIA a 576435 lignes. Voir https://github.com/NVIDIA-Merlin/competitions/blob/main/SIGIR_eCommerce_Challenge_2021/task1_session_based_rec/0-eda/EDA_train_x_test_set.ipynb

**NOTE**: Finalement, le fichier `test/rec_test_phase_1.json` a été créé par l'équipe NVIDIA. L'énoncé de la compétition ne parle pas de données test. Va falloir créer notre fichier test à partir des ensembles de données...

## Analyzing train data

Le code de cette section provient principalement de NVIDIA. Voir https://github.com/NVIDIA-Merlin/competitions/blob/main/SIGIR_eCommerce_Challenge_2021/task1_session_based_rec/0-eda/EDA_train_x_test_set.ipynb.

### Items

In [98]:
train_browsing_items = train_browsing_df['product_sku_hash'].unique()
len(train_browsing_items)

57484

In [100]:
# Avant transformation des colonnes.
train_search_items = train_search_df[~train_search_df['product_skus_hash'].isna()]['product_skus_hash'].explode().unique()
len(train_search_items)

189796

In [114]:
import ast

def convert_str_to_list(x): 
    if pd.isnull(x): 
        return x
    return ast.literal_eval(x)

In [115]:
# Test pour comprendre ce que fait ast.literal_eval(x):
c = ["['1', '2', '3']", None, "['5', '6', '7']"]  # des chaînes représentant des listes
df = pd.DataFrame({'c': c})

print("Avant la conversion :")
print(df)

for col in ['c']: 
    df[col] = df[col].apply(lambda x: convert_str_to_list(x))

print("\nAprès la conversion :")
print(df)

Avant la conversion :
                 c
0  ['1', '2', '3']
1             None
2  ['5', '6', '7']

Après la conversion :
           c
0  [1, 2, 3]
1       None
2  [5, 6, 7]


In [102]:
for col in ['product_skus_hash', 'clicked_skus_hash', 'query_vector']: 
    train_search_df[col] = train_search_df[col].apply(convert_str_to_list)

In [103]:
# Après transformation des colonnes.
train_search_items = train_search_df[~train_search_df['product_skus_hash'].isna()]['product_skus_hash'].explode().unique()
len(train_search_items)

30399

**NOTE** : Différence ici. J'obtiens 30399 tandis que NVIDIA obtiens 189796. Voir https://github.com/NVIDIA-Merlin/competitions/blob/main/SIGIR_eCommerce_Challenge_2021/task1_session_based_rec/0-eda/EDA_train_x_test_set.ipynb

In [15]:
content_items = train_sku_to_content_df['product_sku_hash'].unique()
len(content_items)

66386

In [16]:
len(set(train_browsing_items).difference(set(train_search_items)))

28182

In [17]:
len(set(train_search_items).difference(set(train_browsing_items)))

1097

In [18]:
len(set(train_search_items_clicked).difference(set(train_browsing_items)))

73311

**NOTE** : Grosse différence ici. J'obtiens 73311 tandis que NVIDIA obtiens 100. Dans un Jupyter notebook, on n'est jamais certain quelles cellules ont été exécutées avant... Cet écart provient sûrement de cela.

### Sessions

In [19]:
train_browsing_df['session_id_hash'].nunique()

4934699

In [20]:
len(set(train_browsing_df['session_id_hash'].unique()).difference(set(train_search_df['session_id_hash'].unique())))

4460582

In [21]:
len(set(train_search_df['session_id_hash'].unique()).difference(set(train_browsing_df['session_id_hash'].unique())))

75983

## Comparing train and test data

See https://github.com/NVIDIA-Merlin/competitions/blob/main/SIGIR_eCommerce_Challenge_2021/task1_session_based_rec/0-eda/EDA_train_x_test_set.ipynb

**NOTE** : On ne peut pas réellement comparer puisque le fichier test `json` n'est pas comparable à celui de NVIDIA.

## Processing DATA with pandas

Voir https://github.com/NVIDIA-Merlin/competitions/blob/main/SIGIR_eCommerce_Challenge_2021/task1_session_based_rec/1-preprocessing/ecom_preproc_step1.ipynb

In [22]:
import os

In [23]:
DATA_FOLDER = "/Users/alain/data/SIGIR-ecom-data-challenge/train/"
FILENAME_PATTERN_BROWSING = 'browsing_train.csv'
FILENAME_PATTERN_SEARCH = 'search_train.csv'
FILENAME_PATTERN_SKU = 'sku_to_content.csv'
DATA_PATH_BROWSING = os.path.join(DATA_FOLDER, FILENAME_PATTERN_BROWSING)
DATA_PATH_SEARCH = os.path.join(DATA_FOLDER, FILENAME_PATTERN_SEARCH)
DATA_PATH_SKU = os.path.join(DATA_FOLDER, FILENAME_PATTERN_SKU)
OUTPUT_DIR = "/Users/alain/data/workspace"
!ls $DATA_PATH_BROWSING

/Users/alain/data/SIGIR-ecom-data-challenge/train/browsing_train.csv


In [24]:
MINIMUM_SESSION_LENGTH = 2

### Proprocessing of search tables

In [85]:
# load search data
search = pd.read_csv(DATA_PATH_SEARCH, sep=',')
search.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 819516 entries, 0 to 819515
Data columns (total 5 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   session_id_hash            819516 non-null  object
 1   query_vector               819516 non-null  object
 2   clicked_skus_hash          179495 non-null  object
 3   product_skus_hash          602754 non-null  object
 4   server_timestamp_epoch_ms  819516 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 31.3+ MB


In [73]:
# drop 123 rows where: (clicked_skus_hash != NaN) and (product_skus_hash == NaN)
condition = (search['product_skus_hash'].isnull()) & (~search['clicked_skus_hash'].isnull())
condition.value_counts()

False    819393
True        123
Name: count, dtype: int64

In [86]:
# Add column event_type 
search['event_type'] = 'search'
# Add column 'is_search'
search['is_search'] = 1
search['is_test'] = 0
# drop 123 rows where: (clicked_skus_hash != NaN) and (product_skus_hash == NaN)
condition = (search['product_skus_hash'].isnull()) & (~search['clicked_skus_hash'].isnull())
search = search.drop(index=search[condition].index)

# convert strings to list object 
import ast
def convert_str_to_list(x): 
    if pd.isnull(x): 
        return x
    return ast.literal_eval(x)

for col in ['product_skus_hash', 'clicked_skus_hash', 'query_vector']: 
    search[col] = search[col].progress_apply(lambda x: convert_str_to_list(x))


100%|██████████████████████████████| 819393/819393 [01:17<00:00, 10563.53it/s]


In [75]:
search.info()

<class 'pandas.core.frame.DataFrame'>
Index: 819393 entries, 0 to 819515
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   session_id_hash            819393 non-null  object
 1   query_vector               819393 non-null  object
 2   clicked_skus_hash          179372 non-null  object
 3   product_skus_hash          602754 non-null  object
 4   server_timestamp_epoch_ms  819393 non-null  int64 
 5   event_type                 819393 non-null  object
 6   is_search                  819393 non-null  int64 
 7   is_test                    819393 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 56.3+ MB


**NOTE** : On ne peut pas rouler la cellule ci-dessous puisqu'on n'a pas le fichier `rec_test_phase_1.json`.

In [ ]:
# Add search events from test data
# load test data 
with open('/workspace/rec_test_phase_1.json') as json_file:
    # read the test cases from the provided file
    test_queries = json.load(json_file)
test_df = pd.json_normalize(test_queries, 'query')
test_df['is_test'] = 1
test_search = test_df[['session_id_hash', 'query_vector', 'clicked_skus_hash',
       'product_skus_hash', 'server_timestamp_epoch_ms', 'event_type',
       'is_search', 'is_test']]
test_search = test_search[test_search.is_search==True]

# concat test and train search data
search = pd.concat([search, test_search])
search.reset_index(inplace=True)

### Compute the number of search queries per session

In [76]:
tmp = search.groupby('session_id_hash').size().reset_index()
tmp.columns = ['session_id_hash', 'nb_queries']
search = search.merge(tmp, on='session_id_hash', how='left')

### Include unseen clicked product to impression list

2% of the search events have a clicked item that does not appear in the impression list. ==> We add the missing elements to the impression list.

**NOTE** : Je ne comprends pas comment la cellule ci-dessous implémente le commentaire ci-dessus.

In [78]:
def add_clicked(x): 
    if isinstance(x.clicked_skus_hash, list) and isinstance(x.product_skus_hash, list):
        return list(set(x.product_skus_hash).union(set(x.clicked_skus_hash)))
    return x.product_skus_hash
search['updated_product_skus_hash'] = search.apply(add_clicked, axis=1)